In [ ]:
import pandas as pd
from google.colab import files
import io

# 1. Função para fazer o upload do arquivo
print("Envie o arquivo:")
uploaded = files.upload()

# 2. Carregar o arquivo para uma tabela (DataFrame) do Pandas
# Pegamos o nome do primeiro arquivo enviado (caso você envie apenas um)
file_name = next(iter(uploaded))

try:
    # Tentativa de leitura padrão
    df = pd.read_csv(io.BytesIO(uploaded[file_name]))

    print(f"\nSucesso! O arquivo '{file_name}' foi carregado.")
    print(f"Dimensões da base: {df.shape[0]} linhas e {df.shape[1]} colunas.")

    # 3. Mostrar as primeiras 5 linhas para confirmar se está tudo certo
    print("\nVisualização das primeiras linhas:")
    display(df.head())

except Exception as e:
    print("Ocorreu um erro ao ler o arquivo. Verifique se é um CSV válido.")
    print(e)

Envie o arquivo:


Saving all_in_balanceado.csv to all_in_balanceado.csv

Sucesso! O arquivo 'all_in_balanceado.csv' foi carregado.
Dimensões da base: 1421 linhas e 26 colunas.

Visualização das primeiras linhas:


,id,Per. Ingresso,Curso,Forma de Ingresso,Sigla Cota,Escola Pública?,Coeficiente,Sexo,Idade,Situação Atual do Aluno,...,Aprovado por Nota/Frequência,Reprovado por Nota/Frequência,Crédito Consignado,Reprovado por Nota,Aprovado em Exame de Suficiência,Aprovado por Nota,Reprovado por Frequência,Reprovado em Exame de Suficência,Reprovado,Aprovado por Frequência
0,eg335c,1,Ciência Da Computação,SISU - Sistema de Seleção Unificada,Não cotista,Escola Pública,0.14740,M,27,Desistente/Jubilado,...,1,1,0,4,0,0,0,0,0,0
1,788a5c,1,Ciência Da Computação,SISU - Sistema de Seleção Unificada,Cotista,Escola Pública,0.34150,F,27,Desistente/Jubilado,...,2,0,0,4,0,0,0,0,0,0
2,nh1pjk,1,Ciência Da Computação,SISU - Sistema de Seleção Unificada,Não cotista,Escola Particular,0.25750,M,27,Desistente/Jubilado,...,5,6,0,4,0,0,0,0,0,0
3,31hbpx,1,Ciência Da Computação,SISU - Sistema de Seleção Unificada,Cotista,Escola Pública,0.16430,M,37,Desistente/Jubilado,...,0,9,3,0,0,0,0,0,0,0
4,bfo4ff,1,Ciência Da Computação,SISU - Sistema de Seleção Unificada,Não cotista,Escola Particular,0.15593,M,27,Desistente/Jubilado,...,3,12,0,6,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import pandas as pd

# ================================
# 1. LIMPEZA E CODIFICAÇÃO
# ================================

# Remove coluna ID se existir (garantindo consistência)
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Define coluna alvo
coluna_alvo = 'Situação Atual do Aluno'

# Codificação do alvo (LabelEncoder)
le = LabelEncoder()
y = le.fit_transform(df[coluna_alvo])

# Features
X = df.drop(columns=[coluna_alvo])

# One-hot encoding SEM drop_first (evita perda de categorias e erros no teste)
X = pd.get_dummies(X, drop_first=False)

print("✔️ Dados codificados corretamente!")

# ================================
# 2. SEPARAÇÃO TREINO/TESTE COM ESTRATIFICAÇÃO
# ================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"Treino: {X_train.shape[0]} linhas")
print(f"Teste:  {X_test.shape[0]} linhas")

# Verificação do balanceamento
prop_treino = sum(y_train) / len(y_train)
prop_teste = sum(y_test) / len(y_test)

print(f"Proporção classe 1 (Treino): {prop_treino:.2%}")
print(f"Proporção classe 1 (Teste):  {prop_teste:.2%}")
print("✔️ Estratificação preservada!")

# ================================
# 3. NORMALIZAÇÃO
# ================================

# MinMaxScaler em todas as features numéricas (One-Hot continua igual pois já é 0/1)
scaler = MinMaxScaler()

X_train_norm = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns
)

X_test_norm = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns
)

print("✔️ Normalização concluída!")


✔️ Dados codificados corretamente!
Treino: 994 linhas
Teste:  427 linhas
Proporção classe 1 (Treino): 50.00%
Proporção classe 1 (Teste):  49.88%
✔️ Estratificação preservada!
✔️ Normalização concluída!


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import matplotlib.pyplot as plt

# ============================================
# 1. ALGORITMO SUPERVISIONADO — ÁRVORE DE DECISÃO
# ============================================
print("--- 1. ÁRVORE DE DECISÃO (Supervisionado) ---")

# Modelo (C4.5 ≈ entropy)
arvore = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=5,
    random_state=42
)

# Treinamento
arvore.fit(X_train_norm, y_train)

# Predições
y_pred = arvore.predict(X_test_norm)

# Avaliação
acc = accuracy_score(y_test, y_pred)

print(f"Acurácia: {acc:.2%}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Importância das features
importances = pd.Series(arvore.feature_importances_, index=X_train_norm.columns)
print("\nTop 3 atributos mais importantes:")
print(importances.nlargest(3))

print("\n" + "="*60 + "\n")

# ============================================
# 2. ALGORITMO NÃO SUPERVISIONADO — DBSCAN
# ============================================
print("--- 2. DBSCAN (Não Supervisionado) ---")

# DBSCAN configurado (eps ajustado)
dbscan = DBSCAN(eps=1.5, min_samples=5)

clusters = dbscan.fit_predict(X_train_norm)

# Número de clusters e ruído
n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
n_noise = list(clusters).count(-1)

print(f"Configuração: eps=1.5 | min_samples=5")
print(f"Clusters encontrados: {n_clusters}")
print(f"Ruídos (outliers): {n_noise}")

# Tabela cruzada entre clusters e rótulos reais
df_analise = pd.DataFrame({
    'Cluster_DBSCAN': clusters,
    'Situacao_Real': y_train
})

print("\nTabela Cruzada (Cluster vs Rótulo Real):")
print(pd.crosstab(df_analise['Cluster_DBSCAN'], df_analise['Situacao_Real']))


--- 1. ÁRVORE DE DECISÃO (Supervisionado) ---
Acurácia: 84.54%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       214
           1       0.87      0.81      0.84       213

    accuracy                           0.85       427
   macro avg       0.85      0.85      0.85       427
weighted avg       0.85      0.85      0.85       427


Top 3 atributos mais importantes:
Período do Aluno    0.470380
Idade               0.219876
Coeficiente         0.099316
dtype: float64


--- 2. DBSCAN (Não Supervisionado) ---
Configuração: eps=1.5 | min_samples=5
Clusters encontrados: 3
Ruídos (outliers): 515

Tabela Cruzada (Cluster vs Rótulo Real):
Situacao_Real     0    1
Cluster_DBSCAN          
-1              248  267
 0              249  207
 1                0   17
 2                0    6


In [ ]:
# ============================================================
# EXPORTAÇÃO DO MODELO – GOOGLE COLAB
# ============================================================

import joblib

# ------------------------------------------------------------
# 1. Exportar o Modelo Treinado (Árvore de Decisão)
# ------------------------------------------------------------
joblib.dump(arvore, "modelo_arvore_decisao.pkl")
print("Modelo da Árvore de Decisão exportado: modelo_arvore_decisao.pkl")

# ------------------------------------------------------------
# 2. Exportar o Scaler (MinMaxScaler)
# ------------------------------------------------------------
joblib.dump(scaler, "scaler.pkl")
print("Scaler exportado: scaler.pkl")

# ------------------------------------------------------------
# 3. Exportar o Label Encoder do Alvo
# ------------------------------------------------------------
joblib.dump(le, "label_encoder.pkl")
print("LabelEncoder exportado: label_encoder.pkl")

# ------------------------------------------------------------
# 4. Ver arquivos salvos no diretório atual (opcional)
# ------------------------------------------------------------
import os

print("\nArquivos no diretório atual:")
for f in os.listdir():
    if f.endswith(".pkl"):
        print(" -", f)

print("\nExportação concluída com sucesso!")


Modelo da Árvore de Decisão exportado: modelo_arvore_decisao.pkl
Scaler exportado: scaler.pkl
LabelEncoder exportado: label_encoder.pkl

Arquivos no diretório atual:
 - scaler.pkl
 - label_encoder.pkl
 - modelo_arvore_decisao.pkl

Exportação concluída com sucesso!


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree

# ==============================================================================
# 1. GERAR MATRIZ DE CONFUSÃO (GRÁFICO)
# ==============================================================================
# Calcula a matriz
cm = confusion_matrix(y_test, y_pred_arvore)

# Plota o gráfico colorido
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Desistente', 'Formado'],
            yticklabels=['Desistente', 'Formado'])
plt.xlabel('Previsão do Modelo')
plt.ylabel('Realidade')
plt.title('Matriz de Confusão - Árvore de Decisão')
plt.show()

# ==============================================================================
# 2. DESENHAR A ÁRVORE DE DECISÃO
# ==============================================================================
# Vamos plotar apenas os níveis superiores para ficar legível no relatório
plt.figure(figsize=(20, 10))
plot_tree(arvore,
          feature_names=X_train_norm.columns,
          class_names=['Desistente', 'Formado'],
          filled=True,
          rounded=True,
          fontsize=10,
          max_depth=3) # Mostra apenas as 3 primeiras camadas de decisão
plt.title("Regras de Decisão (Topo da Árvore)")
plt.show()

NameError: name 'y_pred_arvore' is not defined